# Compare v9: r0 vs r1

This notebook is dedicated to export the v10 model to ONNX and keras versiosn. Usually, keras versions is used into the prometheus framework. The ONNX version will be used into the athena framework.

**NOTE**: ONNX is a Microsoft API for inference.

**NOTE**: We will export only models from r2 derivation.

In [1]:
from kolmov import crossval_table, get_color_fader
import saphyra
import numpy as np
import pandas as pd
import collections
import os
import matplotlib
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

Welcome to JupyROOT 6.23/01
Using all sub packages with ROOT dependence


In [2]:
def create_op_dict(op):
    d = {
              op+'_pd_ref'    : "reference/"+op+"_cutbased/pd_ref#0",
              op+'_fa_ref'    : "reference/"+op+"_cutbased/fa_ref#0",
              op+'_sp_ref'    : "reference/"+op+"_cutbased/sp_ref",
              op+'_pd_val'    : "reference/"+op+"_cutbased/pd_val#0",
              op+'_fa_val'    : "reference/"+op+"_cutbased/fa_val#0",
              op+'_sp_val'    : "reference/"+op+"_cutbased/sp_val",
              op+'_pd_op'     : "reference/"+op+"_cutbased/pd_op#0",
              op+'_fa_op'     : "reference/"+op+"_cutbased/fa_op#0",
              op+'_sp_op'     : "reference/"+op+"_cutbased/sp_op",

              # Counts
              op+'_pd_ref_passed'    : "reference/"+op+"_cutbased/pd_ref#1",
              op+'_fa_ref_passed'    : "reference/"+op+"_cutbased/fa_ref#1",
              op+'_pd_ref_total'     : "reference/"+op+"_cutbased/pd_ref#2",
              op+'_fa_ref_total'     : "reference/"+op+"_cutbased/fa_ref#2",
              op+'_pd_val_passed'    : "reference/"+op+"_cutbased/pd_val#1",
              op+'_fa_val_passed'    : "reference/"+op+"_cutbased/fa_val#1",
              op+'_pd_val_total'     : "reference/"+op+"_cutbased/pd_val#2",
              op+'_fa_val_total'     : "reference/"+op+"_cutbased/fa_val#2",
              op+'_pd_op_passed'     : "reference/"+op+"_cutbased/pd_op#1",
              op+'_fa_op_passed'     : "reference/"+op+"_cutbased/fa_op#1",
              op+'_pd_op_total'      : "reference/"+op+"_cutbased/pd_op#2",
              op+'_fa_op_total'      : "reference/"+op+"_cutbased/fa_op#2",
    }
    return d

tuned_info = collections.OrderedDict( {
              # validation
              "max_sp_val"      : 'summary/max_sp_val',
              "max_sp_pd_val"   : 'summary/max_sp_pd_val#0',
              "max_sp_fa_val"   : 'summary/max_sp_fa_val#0',
              # Operation
              "max_sp_op"       : 'summary/max_sp_op',
              "max_sp_pd_op"    : 'summary/max_sp_pd_op#0',
              "max_sp_fa_op"    : 'summary/max_sp_fa_op#0',
              } )

tuned_info.update(create_op_dict('tight'))
tuned_info.update(create_op_dict('medium'))
tuned_info.update(create_op_dict('loose'))
tuned_info.update(create_op_dict('vloose'))

In [3]:
etbins = [15, 20, 30, 40, 50, 1000000]
etabins = [0.0, 0.8, 1.37, 1.54, 2.37, 2.50]

## 1) Reading all tunings:

In [5]:
cv_v8  = crossval_table( tuned_info, etbins = etbins , etabins = etabins )
cv_v8.fill( '/Volumes/castor/tuning_data/Zee/v8/r0/*/*/*.gz', 'v8')

2021-03-05 21:22:27,924 | Py.crossval_table                       INFO Reading file for v8 tag from /Volumes/castor/tuning_data/Zee/v8/r0/*/*/*.gz
2021-03-05 21:22:27,925 | Py.crossval_table                       INFO There are 9998 files for this task...
2021-03-05 21:22:27,925 | Py.crossval_table                       INFO Filling the table... 
2021-03-05 21:26:36,796 | Py.crossval_table                       INFO End of fill step, a pandas DataFrame was created...


In [6]:
cv_r0  = crossval_table( tuned_info, etbins = etbins , etabins = etabins )
cv_r0.fill( '/Volumes/castor/tuning_data/Zee/v9/r0/*/*/*.gz', 'v9-nofused')

2021-03-05 21:27:45,366 | Py.crossval_table                       INFO Reading file for v9-nofused tag from /Volumes/castor/tuning_data/Zee/v9/r0/*/*/*.gz
2021-03-05 21:27:45,366 | Py.crossval_table                       INFO There are 2500 files for this task...
2021-03-05 21:27:45,366 | Py.crossval_table                       INFO Filling the table... 
2021-03-05 21:29:02,459 | Py.crossval_table                       INFO End of fill step, a pandas DataFrame was created...


In [7]:
cv_r1  = crossval_table( tuned_info, etbins = etbins , etabins = etabins )
cv_r1.fill( '/Volumes/castor/tuning_data/Zee/v9/r1/*/*/*.gz', 'v9')

2021-03-05 21:29:03,205 | Py.crossval_table                       INFO Reading file for v9 tag from /Volumes/castor/tuning_data/Zee/v9/r1/*/*/*.gz
2021-03-05 21:29:03,205 | Py.crossval_table                       INFO There are 2500 files for this task...
2021-03-05 21:29:03,205 | Py.crossval_table                       INFO Filling the table... 
2021-03-05 21:30:22,354 | Py.crossval_table                       INFO End of fill step, a pandas DataFrame was created...


## 2) Compare r1 and r2:

In [8]:
best_inits_v8 = cv_v8.filter_inits("max_sp_val")
best_sorts_v8 = cv_v8.filter_sorts( best_inits_v8.loc[best_inits_v8.model_idx==3] , 'max_sp_op')
best_inits_v8.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,max_sp_val,max_sp_pd_val,...,vloose_pd_ref_total,vloose_fa_ref_total,vloose_pd_val_passed,vloose_fa_val_passed,vloose_pd_val_total,vloose_fa_val_total,vloose_pd_op_passed,vloose_fa_op_passed,vloose_pd_op_total,vloose_fa_op_total
223,v8,0,0,0,0,3,/Volumes/castor/tuning_data/Zee/v8/r0/user.jod...,0,0.978092,0.983378,...,232819,187639,23016,690,23282,18764,230153,7114,232819,187639
24,v8,0,0,0,1,2,/Volumes/castor/tuning_data/Zee/v8/r0/user.jod...,0,0.975835,0.983464,...,232819,187639,23015,737,23282,18764,230152,7025,232819,187639
168,v8,0,0,0,2,5,/Volumes/castor/tuning_data/Zee/v8/r0/user.jod...,0,0.975044,0.983163,...,232819,187639,23016,754,23282,18764,230153,6959,232819,187639
290,v8,0,0,0,3,6,/Volumes/castor/tuning_data/Zee/v8/r0/user.jod...,0,0.976306,0.984838,...,232819,187639,23015,710,23282,18764,230153,6957,232819,187639
343,v8,0,0,0,4,8,/Volumes/castor/tuning_data/Zee/v8/r0/user.jod...,0,0.977536,0.983335,...,232819,187639,23015,669,23282,18764,230153,6952,232819,187639


In [9]:
best_inits_r0 = cv_r0.filter_inits("max_sp_val")
best_sorts_r0 = cv_r0.filter_sorts( best_inits_r0 , 'max_sp_op')
best_inits_r0.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,max_sp_val,max_sp_pd_val,...,vloose_pd_ref_total,vloose_fa_ref_total,vloose_pd_val_passed,vloose_fa_val_passed,vloose_pd_val_total,vloose_fa_val_total,vloose_pd_op_passed,vloose_fa_op_passed,vloose_pd_op_total,vloose_fa_op_total
34,v9-nofused,0,0,0,0,4,/Volumes/castor/tuning_data/Zee/v9/r0/user.jod...,0,0.982898,0.986213,...,232819,187639,23015,443,23282,18764,230154,4558,232819,187639
2,v9-nofused,0,0,0,1,0,/Volumes/castor/tuning_data/Zee/v9/r0/user.jod...,0,0.982252,0.986685,...,232819,187639,23015,473,23282,18764,230153,4565,232819,187639
19,v9-nofused,0,0,0,2,6,/Volumes/castor/tuning_data/Zee/v9/r0/user.jod...,0,0.980962,0.984366,...,232819,187639,23015,533,23282,18764,230153,4574,232819,187639
25,v9-nofused,0,0,0,3,1,/Volumes/castor/tuning_data/Zee/v9/r0/user.jod...,0,0.981744,0.985826,...,232819,187639,23014,491,23282,18764,230154,4694,232819,187639
37,v9-nofused,0,0,0,4,2,/Volumes/castor/tuning_data/Zee/v9/r0/user.jod...,0,0.982666,0.988961,...,232819,187639,23017,440,23282,18764,230155,4697,232819,187639


In [10]:
best_inits_r1 = cv_r1.filter_inits("max_sp_val")
best_sorts_r1 = cv_r1.filter_sorts( best_inits_r1 , 'max_sp_op')
best_inits_r1.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,max_sp_val,max_sp_pd_val,...,vloose_pd_ref_total,vloose_fa_ref_total,vloose_pd_val_passed,vloose_fa_val_passed,vloose_pd_val_total,vloose_fa_val_total,vloose_pd_op_passed,vloose_fa_op_passed,vloose_pd_op_total,vloose_fa_op_total
23,v9,0,0,0,0,3,/Volumes/castor/tuning_data/Zee/v9/r1/user.jod...,0,0.982859,0.985439,...,232819,187639,23015,445,23282,18764,230153,4476,232819,187639
11,v9,0,0,0,1,9,/Volumes/castor/tuning_data/Zee/v9/r1/user.jod...,0,0.982398,0.988102,...,232819,187639,23015,449,23282,18764,230153,4483,232819,187639
21,v9,0,0,0,2,8,/Volumes/castor/tuning_data/Zee/v9/r1/user.jod...,0,0.980846,0.985311,...,232819,187639,23015,527,23282,18764,230152,4503,232819,187639
29,v9,0,0,0,3,5,/Volumes/castor/tuning_data/Zee/v9/r1/user.jod...,0,0.981896,0.989992,...,232819,187639,23016,469,23282,18764,230154,4498,232819,187639
36,v9,0,0,0,4,1,/Volumes/castor/tuning_data/Zee/v9/r1/user.jod...,0,0.982538,0.988704,...,232819,187639,23016,442,23282,18764,230152,4644,232819,187639


### 2.1) Make table:

In [11]:
best_inits = pd.concat( [best_inits_v8, best_inits_r0, best_inits_r1] )
for op in ['tight','medium','loose','vloose']:
    cv_r1.dump_beamer_table( best_inits ,  [op], 'tuning_v9_r0_vs_r1_'+op, title = op+' Tunings (v9): r0 vs r1', 
                            tags = ['v8','v9-nofused','v9'])

2021-03-05 21:32:03,342 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file tuning_v9_r0_vs_r1_tight.pdf latex code...
2021-03-05 21:32:09,651 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file tuning_v9_r0_vs_r1_medium.pdf latex code...
2021-03-05 21:32:15,211 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file tuning_v9_r0_vs_r1_loose.pdf latex code...
2021-03-05 21:32:20,805 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file tuning_v9_r0_vs_r1_vloose.pdf latex code...
